# 07. PyTorch Experiment Tracking

We've trained a few models now on the journey to making FoodVision.

And so far we've kept track of them via `Python Dictionaries` or just comparing them by the metric print outs during training.

**but, what if we want to run a dozen (or more) different models at once?**

There's a better way to keep track of the models.

It's called `Experiment Tracking`.

and experiment tracking is very important and integral to machine learning.

Since it is an important topic, you can consider this notebook your first milestone project.

So welcome to `Milestone Project 1: FoodVision Mini Experiment Tracking`

we are going to see `how we can track our machine learning experiments`

## What is experiment tracking?
If the number of experiments we run starts to increase, this naive way of tracking the results with print out and dictionaries could get out of hand.

So, if we're following the machine learning practitioner's motto of experiment, experiment, experiment, we'll want to track them all.

![](07_experiment_tracking.png)

Now, when we have this many models and tracking their results, we'll start to notice how quickly it can get out of hand.

## Different ways to track machine learning experiments.

There are many different ways to track machine learning experiments as there is experiments to run.

Few of them are.
| Method | Setup | Pros | Cons | Cost |
| ------ | ----- | ---- | ---- | ---- |
| PyThon dictionaries, csv files, printouts | None | Easy to setup, runs in pure python | Hard to keep track of large numbers of experiments | Free |
| **Tensorboard** | minimal, install `tensorboard` | Extensions built into PyTorch, widely recognized and used, easily scales. | User-experience not as nice as other options. | Free |
| **Weights and Biases Experiment Tracking** | Minimal, install `wandb`, make an account | Incredible user experience, make experiments public, tracks almost anything. | Requires external resource outside of PyTorch. | Free for personal use |
| MLFlow | Minimal, install `mlflow` and starting tracking | Fully open-source MLOps lifecycle management, many integrations. | Little bit harder to setup a remote tracking server than other services. | Free |

![](expriment_tracking_ways.png)

**Note:** There are more than these, which we can search online later.

## What we're going to cover

we're going to run several different modeling experiments with various levels of data, model size, and training time to try and improve on FoodVision.

We'll look at `TensorBoard` to track our experiments. Cause it is tighly integrated with PyTorch and it is widely used.

However, the principles we're going to cover are similar across all of the other tools for experiment tracking.
